In [67]:
# ensure all examine data.json is in traj data.json

rd = '/checkpoint/apratik/data_dec'

import os
import json

def check_subset(a, b, ctr, eid):
    possible_starts = []
    while True:
        # print(ctr, len(a.keys()))
        if ctr >= len(a.keys()):
            return ctr, True
        if a[str(ctr)] == b['0']:
            possible_starts.append(ctr)
            # print(f'{ctr} key is equal to key 0')
            # print(a[str(ctr)], b['0'])
            while ctr < len(a.keys()) and a[str(ctr)] == b['0']:
                possible_starts.append(ctr)
                ctr += 1
            ctr -= 1
            break
        ctr += 1
    
    # print(ctr)
    
        
    is_valid = False
    for ctr in possible_starts:
        s = ctr
        isv = True
        for k, v in b.items():
            if ctr >= len(a.keys()) or b[k] != a[str(ctr)]:
                # print(f'{k, b[k]} != {ctr, a[str(ctr)]}')
                isv = False
                break
            ctr += 1
        is_valid |= isv
        if is_valid:
            # print(f'start idx {s} for {eid}')
            break
    
    return ctr, is_valid

def check_traj(traj_path):
    # load all data json
    with open(os.path.join(traj_path, 'data.json'), 'r') as f:
        base_data = json.load(f)
        # print(base_data)
        
    base_ctr = 0
    # print(f'{len([x for x in os.listdir(traj_path) if x.isdigit()])} examines in {traj_path}')
    for x in os.listdir(traj_path):
        if x.isdigit():
            with open(os.path.join(traj_path, x, 'data.json'), 'r') as f:
                cur = json.load(f)
                # if x == '11':
                #     print(base_data, cur)
                # now check that all cur is in base_data and increment base_ctr
                base_ctr, is_valid = check_subset(base_data, cur, base_ctr, x)
                assert is_valid, f'{x} not found in base poses'
                                
traj_path = os.path.join(rd, 'straightline', '0')
check_traj(traj_path)

In [66]:
def check_root(heu):
    heur = os.path.join(rd, heu)
    valid_traj = []
    total_traj = 0
    for x in os.listdir(heur):
        if x.isdigit():
            total_traj += 1
            try:
                check_traj(os.path.join(heur, x))
                valid_traj.append(x)
            except AssertionError as ex:
                print(ex)
    assert total_traj == len(valid_traj), f'Only {len(valid_traj)} out of {total_traj}'
    # print(f'total_traj {total_traj}, valid_traj {len(valid_traj)}')
    print(f'All trajectories valid for {heu}')

  
og_list = [
    'circle', 
    'circle_noisy2',
    'baseline', 
    'baseline_noisy2',
    'straightline', 
    'st_noisy2',
]

for x in og_list:
    check_root(x)
                

All trajectories valid for circle
All trajectories valid for circle_noisy2
All trajectories valid for baseline
All trajectories valid for baseline_noisy2
All trajectories valid for straightline
All trajectories valid for st_noisy2
